In [1]:
import sys
import os
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
# Add scripts directory to path for utils import
sys.path.append('/home/tobamo/analize/project-tobamo/analysis/model/scripts')
from utils import (
    remove_duplicates, 
    pairwise_refs, 
    prep_df, 
    make_distance_matrix, 
    make_dendrograms, 
    make_cluster_df, 
    select_species, 
    make_dendrogram_selected_species
)

In [2]:
# domain scientists provided sequences from NCBI accessed 2025-01-20, for all Virgaviridae genera
virga_paths = glob('/home/tobamo/analize/project-tobamo/analysis/data/domain_sci_input/ncbi_virgaviridae_20250120/*.fasta')

new_virga_dict = {}
for path in virga_paths:
    genus = path.split('/')[-1].split('RNA1_')[0]
    # Store SeqRecord objects instead of just IDs
    sequences = list(SeqIO.parse(path, 'fasta'))
    new_virga_dict[genus] = sequences

In [3]:
# Display comprehensive sequence statistics for each genus
print("NCBI VIRGAVIRIDAE SEQUENCES (2025-01-20) - GENUS SUMMARY")
print("=" * 80)
print(f"{'Genus':<15} {'Count':<8} {'Min Length':<12} {'Max Length':<12} {'Avg Length':<12}")
print("-" * 80)

total_sequences = 0
for genus, seqs in sorted(new_virga_dict.items()):
    if seqs:  # Only show genera with sequences
        lengths = [len(seq.seq) for seq in seqs]
        count = len(seqs)
        min_len = min(lengths)
        max_len = max(lengths)
        avg_len = int(sum(lengths)/len(lengths))
        
        print(f"{genus.upper():<15} {count:<8} {min_len:<12,} {max_len:<12,} {avg_len:<12,}")
        total_sequences += count

print("-" * 80)
print(f"{'TOTAL':<15} {total_sequences:<8}")
print("\nReady for representative selection via hierarchical clustering...")

NCBI VIRGAVIRIDAE SEQUENCES (2025-01-20) - GENUS SUMMARY
Genus           Count    Min Length   Max Length   Avg Length  
--------------------------------------------------------------------------------
FUROVIRUS       34       6,878        7,226        7,119       
GORAVIRUS       5        4,490        5,519        5,062       
HORDEIVIRUS     17       3,738        3,873        3,778       
PECLUVIRUS      4        5,841        5,897        5,872       
POMOVIRUS       51       5,600        6,170        5,967       
TOBAMOVIRUS     1142     4,679        6,794        6,359       
TOBRAVIRUS      34       6,703        7,073        6,807       
--------------------------------------------------------------------------------
TOTAL           1287    

Ready for representative selection via hierarchical clustering...


In [4]:
# Create output directories
os.makedirs('results/reference_selection/virgaviridae', exist_ok=True)
os.makedirs('results/reference_selection/pairwise_aln', exist_ok=True)

for genus, ref_names in new_virga_dict.items():
    print(f"\nProcessing genus: {genus.upper()}")
    
    fin_path = f'../../data/domain_sci_input/ncbi_virgaviridae_20250120/{genus}RNA1_sequences.fasta'
    
    # Check if fasta file exists
    if not os.path.exists(fin_path):
        print(f"File {fin_path} does not exist. Skipping {genus}...")
        continue

    # Step 1: Remove duplicated sequences
    fout_path = f'results/reference_selection/virgaviridae/{genus}_RNA1_unique.fasta'
    mapper_path = f'results/reference_selection/virgaviridae/{genus}_RNA1_mapper.txt'

    unique_seqs = remove_duplicates(fin_path, fout_path, mapper_path)
    
    # Step 2: Pairwise alignment within genus only
    df_out = f'results/reference_selection/pairwise_aln/pairwise_aln_{genus}_RNA1_sequences.csv'
    
    if os.path.exists(df_out):
        print(f"Pairwise alignment file already exists. Skipping...")
    else:
        print(f"Calculating within-genus pairwise alignments...")
        dff = pairwise_refs(fout_path)
        dff.to_csv(df_out, index=False)
    
    print(f"Completed processing for {genus}")


Processing genus: POMOVIRUS
    Total records: 51
    Unique sequences: 48
    Duplicates removed: 3
Pairwise alignment file already exists. Skipping...
Completed processing for pomovirus

Processing genus: PECLUVIRUS
    Total records: 4
    Unique sequences: 3
    Duplicates removed: 1
Pairwise alignment file already exists. Skipping...
Completed processing for pecluvirus

Processing genus: FUROVIRUS
    Total records: 34
    Unique sequences: 28
    Duplicates removed: 6
Pairwise alignment file already exists. Skipping...
Completed processing for furovirus

Processing genus: GORAVIRUS
    Total records: 5
    Unique sequences: 3
    Duplicates removed: 2
Pairwise alignment file already exists. Skipping...
Completed processing for goravirus

Processing genus: TOBAMOVIRUS
    Total records: 1142
    Unique sequences: 1054
    Duplicates removed: 88
Pairwise alignment file already exists. Skipping...
Completed processing for tobamovirus

Processing genus: TOBRAVIRUS
    Total records:

In [5]:
# Load original reference sequences and find overlap with new NCBI sequences
print("Finding overlap between original references and new NCBI sequences...")

# Load original reference sequences
ori = SeqIO.to_dict(SeqIO.parse('/home/tobamo/analize/model-tobamo/data/genome_clean_.fasta', 'fasta'))

# Create combined FASTA from new sequences (if not already exists)
output_fasta_path = '/home/tobamo/analize/project-tobamo/analysis/data/domain_sci_input/combined_RNA1_20250120.fasta'
if not os.path.exists(output_fasta_path):
    print("Creating combined FASTA file from all new sequences...")
    all_new_seqs = []
    for genus, seqs in new_virga_dict.items():
        all_new_seqs.extend(seqs)
    
    with open(output_fasta_path, 'w') as f:
        SeqIO.write(all_new_seqs, f, 'fasta')

# Load new sequences
new = SeqIO.to_dict(SeqIO.parse(output_fasta_path, 'fasta'))

# Extract sequences and their corresponding IDs from both dictionaries
ori_seqs2 = {str(record.seq): record.id for record in ori.values()}
new_seqs2 = {str(record.seq): record.id for record in new.values()}

# Find identical sequences by DNA content
identical_seqs2 = set(ori_seqs2.keys()).intersection(new_seqs2.keys())

# Get all IDs (both original and new) that correspond to identical sequences
identical_ids = [(ori_seqs2[seq], new_seqs2[seq]) for seq in identical_seqs2]
flattened_identical_ids = [id for pair in identical_ids for id in pair]
all_ids_of_original_seqs = list(set(flattened_identical_ids))

# Create length-prefixed versions to match clustering format
length_mapper_ori = [f"{len(record.seq)}_{record.id}" for record in ori.values()]
length_mapper_new = [f"{len(record.seq)}_{record.id}" for record in new.values()]
len_ids = length_mapper_ori + length_mapper_new

# Filter to get only IDs that correspond to original sequences
all_ids_of_original_seqs_ = [id for id in len_ids if any(seq in id for seq in all_ids_of_original_seqs)]

print(f"Found {len(identical_seqs2)} identical sequences between original and new datasets")
print(f"Total original sequence IDs with length prefix: {len(all_ids_of_original_seqs_)}")

Finding overlap between original references and new NCBI sequences...
Found 68 identical sequences between original and new datasets
Total original sequence IDs with length prefix: 140
Found 68 identical sequences between original and new datasets
Total original sequence IDs with length prefix: 140


In [6]:
for genus in new_virga_dict.keys():
    print(f"Processing {genus}...")
    pairwise_df_path = f'results/reference_selection/pairwise_aln/pairwise_aln_{genus}_RNA1_sequences.csv'

    # Check if file exists
    if not os.path.exists(pairwise_df_path):
        print(f"File {pairwise_df_path} does not exist. Skipping {genus}...")
        continue
    
    try:
        selected_col = 'identity_score'
        pairwise_df = pd.read_csv(pairwise_df_path)
        length_mapper = {seq.id: len(seq.seq) for seq in SeqIO.parse(f'results/reference_selection/virgaviridae/{genus}_RNA1_unique.fasta', 'fasta')}

        print(f"Processing {selected_col}...")
        df = prep_df(pairwise_df, length_mapper, selected_col)
        matrix, res, idx = make_distance_matrix(df, selected_col)
        linkage_matrix = make_dendrograms(matrix, res, selected_col, genus)
        cluster_df = make_cluster_df(linkage_matrix, idx, selected_col, genus)
        distance_threshold = cluster_df.loc[cluster_df['identity'] >= 0.9, 'distance'].max()
        print(f"Distance threshold for {genus} RNA1 based on {selected_col}: {distance_threshold}")
        # Select one species per cluster
        selected_species_list = select_species(linkage_matrix, distance_threshold, idx, all_ids_of_original_seqs_, selected_col, genus)
        # Make dendrogram with selected species
        make_dendrogram_selected_species(matrix, res, selected_col, genus, selected_species_list, distance_threshold)
        print(f"Successfully completed processing for {genus}")
        
    except ValueError as e:
        if "negative distances" in str(e):
            print(f"WARNING: Skipping {genus} due to negative distances in linkage matrix: {e}")
            continue
        else:
            print(f"ERROR: Unexpected ValueError for {genus}: {e}")
            raise
    except Exception as e:
        print(f"ERROR: Unexpected error processing {genus}: {e}")
        print(f"Continuing with next genus...")
        continue

Processing pomovirus...
Processing identity_score...
Distance threshold for pomovirus RNA1 based on identity_score: 0.024092354023757756
Successfully completed processing for pomovirus
Processing pecluvirus...
Processing identity_score...
Distance threshold for pecluvirus RNA1 based on identity_score: nan
Successfully completed processing for pomovirus
Processing pecluvirus...
Processing identity_score...
Distance threshold for pecluvirus RNA1 based on identity_score: nan
Successfully completed processing for pecluvirus
Processing furovirus...
Processing identity_score...
Processing goravirus...
Processing identity_score...
Distance threshold for goravirus RNA1 based on identity_score: nan
Successfully completed processing for pecluvirus
Processing furovirus...
Processing identity_score...
Processing goravirus...
Processing identity_score...
Distance threshold for goravirus RNA1 based on identity_score: nan
Successfully completed processing for goravirus
Processing tobamovirus...
Succe